In [2]:
import torchtext
torchtext.__version__

'0.4.0'

In [1]:
from data.make_dataset import iterators_and_fields
from entity.train_pipeline_params import read_training_pipeline_params

In [2]:
data_params = read_training_pipeline_params("../configs/train_config.yml").dataparams

In [3]:
data_params

DataParams(path_to_data='../data/raw/data_little.txt', file_format='tsv', split_ratio=[0.8, 0.15, 0.05], min_word_freq=3, batch_size=128)

In [4]:
vars(data_params)

{'path_to_data': '../data/raw/data_little.txt',
 'file_format': 'tsv',
 'split_ratio': [0.8, 0.15, 0.05],
 'min_word_freq': 3,
 'batch_size': 128}

In [5]:
data_params.path_to_data

'../data/raw/data_little.txt'

In [6]:
import sys
import logging

import pynndescent.distances

from data.make_dataset import iterators_and_fields
from entity.train_pipeline_params import read_training_pipeline_params
from models.base_seq2seq import base_seq2seq
import click
from train_model import train_model
import torch
import torch.nn as nn

In [7]:
logger = logging.getLogger(__name__)
handler = logging.StreamHandler(sys.stdout)
logger.setLevel(logging.INFO)
logger.addHandler(handler)

In [8]:
config_path = "../configs/train_config.yml"
all_params = read_training_pipeline_params(config_path)
all_params

TrainingPipelineParams(dataparams=DataParams(path_to_data='../data/raw/data_little.txt', file_format='tsv', split_ratio=[0.8, 0.15, 0.05], min_word_freq=3, batch_size=128), seq2seqparams=Seq2SeqParams(enc_emb_dim=128, dec_emb_dim=128, enc_hid_dim=256, dec_hid_dim=256, enc_dropout=0.15, dec_dropout=0.15, teacher_forcing_ratio=0.35, n_layers=2, save_path='./src/models/baseline.pt'), trainparams=TrainParams(clip=1, num_epochs=10))

In [10]:
data_params_dict = vars(all_params.dataparams)
data, fields = iterators_and_fields(**data_params_dict)
train_iterator, valid_iterator, test_iterator = data
SRC, TRG = fields
base_model_params = vars(all_params.seq2seqparams)
logger.info("Try build the baseline model...")
base_model, path = base_seq2seq(489, 445, **base_model_params)
logger.info("baseline loaded!!!")
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(base_model.parameters())
train_params = all_params.trainparams

Loading dataset from ../data/raw/data_little.txt...
Loading from ../data/raw/data_little.txt finished
Split the data to train, valid and test...
Number of train, valid and test examples: (563, 106, 35)
Build the vocabularis...
Vocabs is built!
The number of unique tokens in source (ru) vocabulary: 480
The number of unique tokens in target (en) vocabulary: 442
Try to build iterators...
Iterators prepared!!!
Try build the baseline model...
baseline loaded!!!


In [11]:
base_model.load_state_dict(torch.load("models/baseline.pt"))

<All keys matched successfully>

In [23]:
import tqdm
for i, batch in tqdm.tqdm(enumerate(train_iterator)):
    src = batch.src
    trg = batch.trg
    break

0it [00:00, ?it/s]


In [15]:
from utils import get_text

In [24]:
output = base_model(src, trg)
output.shape, src.shape, trg.shape

(torch.Size([35, 51, 445]), torch.Size([40, 51]), torch.Size([35, 51]))

In [25]:
output_rav = output[1:].view(-1, output.shape[-1])
trg_rav = trg[1:].view(-1)
output.shape, src.shape, trg.shape

(torch.Size([35, 51, 445]), torch.Size([40, 51]), torch.Size([35, 51]))

In [26]:
def remove_tech_tokens(mystr, tokens_to_remove=['<eos>', '<sos>', '<unk>', '<pad>']):
    return [x for x in mystr if x not in tokens_to_remove]


def get_text(x, TRG_vocab):
    text = [TRG_vocab.itos[token] for token in x]
    try:
        end_idx = text.index('<eos>')
        text = text[:end_idx]
    except ValueError:
        pass
    text = remove_tech_tokens(text)
    if len(text) < 1:
        text = []
    return text

In [27]:
trg.cpu().numpy().T.shape

(51, 35)

In [35]:
original_text = [get_text(x, TRG.vocab) for x in trg.cpu().numpy().T]
print(original_text[0])

['at', 'apartment', 'casa', 'you', 'will', 'find', 'a', 'garden', '.']


In [37]:
trg.cpu().numpy().T.shape

(128, 50)

In [37]:
import numpy as np
np.argmax(output[1:].detach().cpu().numpy(), axis = -1).T.shape

(51, 34)